# Linear Regression with Spark

In [1]:
import ibmos2spark

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190608195437-0000
KERNEL_ID = eef0faa2-24b2-4f65-a547-2bb6cb624e11


In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
configuration_name = 'os_45b77029797b4322b732418ac53a105b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

df = spark.read.parquet(cos.url('hmp.parquet', 'advancedatasci-donotdelete-pr-bvizfkp0o2ej2y'))

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 27| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 23| 51| 38|Brush_teeth|Accelerometer-201...|
| 25| 51| 39|Brush_teeth|Accelerometer-201...|
| 26| 51| 39|Brush_teeth|Accelerometer-201...|
| 25| 52| 39|Brush_teeth|Accelerometer-201...|
| 25| 50| 38|Brush_teeth|Accelerometer-201...|
| 26| 49| 39|Brush_teeth|Accelerometer-201...|
| 24| 48| 39|Brush_teeth|Accelerometer-201...|
| 24| 47| 39|Brush_teeth|Accelerometer-201...|
| 23| 47| 39|Brush_teeth|Accelerometer-201...|
| 20| 48| 39|Brush_teeth|Accelerometer-201...|
| 21| 47| 40|Brush_teeth|Accelerometer-201...|
| 21| 47| 42|

In [4]:
df.createOrReplaceTempView('df')

In [5]:
df_energy = spark.sql("""

select sqrt(sum(X*X)+sum(Y*Y)+sum(Z*Z)) as label, class from df group by class

""")

df_energy.createOrReplaceTempView('df_energy')

In [6]:
df_join = spark.sql("""

select * from df inner join df_energy on df.class = df_energy.class

""")

df_join.show()

+---+---+---+-----------+--------------------+-----------------+-----------+
|  x|  y|  z|      class|              source|            label|      class|
+---+---+---+-----------+--------------------+-----------------+-----------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 27| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 23| 51| 38|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 25| 51| 39|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|
| 26| 51| 39|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

vectorAssembler = VectorAssembler(inputCols = ["x", "y", "z"],
                                 outputCol = "features")
normalizer = Normalizer(inputCol = "features", outputCol = "features_norm", p = 1.0)

In [8]:
from pyspark.ml.regression import LinearRegression

In [9]:
lr = LinearRegression(maxIter = 10, regParam = 0.3, elasticNetParam = 0.8)

In [10]:
from pyspark.ml import Pipeline

In [11]:
pipeline = Pipeline(stages = [vectorAssembler, normalizer, lr])

In [12]:
model = pipeline.fit(df_join)

In [13]:
prediction = model.transform(df_join)

In [14]:
prediction.show()

+---+---+---+-----------+--------------------+-----------------+-----------+----------------+--------------------+------------------+
|  x|  y|  z|      class|              source|            label|      class|        features|       features_norm|        prediction|
+---+---+---+-----------+--------------------+-----------------+-----------+----------------+--------------------+------------------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[26.0,51.0,36.0]|[0.23008849557522...|11260.914132330869|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[26.0,51.0,36.0]|[0.23008849557522...|11260.914132330869|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[26.0,51.0,37.0]|[0.22807017543859...|11332.369820764736|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|11785.39634462923|Brush_teeth|[25.0,51.0,37.0]|[0.22123893805309...| 11313.99441223891|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|11785.3963446292

# Logistic Regression with Spark

In [15]:
configuration_name = 'os_45b77029797b4322b732418ac53a105b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

df = spark.read.parquet(cos.url('hmp.parquet', 'advancedatasci-donotdelete-pr-bvizfkp0o2ej2y'))

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 27| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 23| 51| 38|Brush_teeth|Accelerometer-201...|
| 25| 51| 39|Brush_teeth|Accelerometer-201...|
| 26| 51| 39|Brush_teeth|Accelerometer-201...|
| 25| 52| 39|Brush_teeth|Accelerometer-201...|
| 25| 50| 38|Brush_teeth|Accelerometer-201...|
| 26| 49| 39|Brush_teeth|Accelerometer-201...|
| 24| 48| 39|Brush_teeth|Accelerometer-201...|
| 24| 47| 39|Brush_teeth|Accelerometer-201...|
| 23| 47| 39|Brush_teeth|Accelerometer-201...|
| 20| 48| 39|Brush_teeth|Accelerometer-201...|
| 21| 47| 40|Brush_teeth|Accelerometer-201...|
| 21| 47| 42|

In [16]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [17]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

indexer = StringIndexer(inputCol = "class", outputCol = "label")

vectorAssembler = VectorAssembler(inputCols = ["x", "y", "z"],
                                 outputCol = "features")

normalizer = Normalizer(inputCol = "features", outputCol = "features_norm", p = 1.0)

In [18]:
from pyspark.ml.classification import LogisticRegression

In [60]:
lr = LogisticRegression(maxIter = 10, regParam = 0.3, elasticNetParam = 0.8)

In [62]:
from pyspark.ml import Pipeline

In [63]:
pipeline = Pipeline(stages = [indexer, vectorAssembler, normalizer, lr])

In [64]:
model = pipeline.fit(df_train)

In [65]:
prediction = model.transform(df_train)

In [66]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [67]:
eval = MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')

In [69]:
eval.evaluate(prediction)

0.7915369679785497

In [70]:
model = pipeline.fit(df_test)

In [71]:
prediction = model.transform(df_test)

In [72]:
eval.evaluate(prediction)

0.7158023184868822

# SVM Using Apache SparkML

In [73]:
configuration_name = 'os_45b77029797b4322b732418ac53a105b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

df = spark.read.parquet(cos.url('hmp.parquet', 'advancedatasci-donotdelete-pr-bvizfkp0o2ej2y'))

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 27| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 23| 51| 38|Brush_teeth|Accelerometer-201...|
| 25| 51| 39|Brush_teeth|Accelerometer-201...|
| 26| 51| 39|Brush_teeth|Accelerometer-201...|
| 25| 52| 39|Brush_teeth|Accelerometer-201...|
| 25| 50| 38|Brush_teeth|Accelerometer-201...|
| 26| 49| 39|Brush_teeth|Accelerometer-201...|
| 24| 48| 39|Brush_teeth|Accelerometer-201...|
| 24| 47| 39|Brush_teeth|Accelerometer-201...|
| 23| 47| 39|Brush_teeth|Accelerometer-201...|
| 20| 48| 39|Brush_teeth|Accelerometer-201...|
| 21| 47| 40|Brush_teeth|Accelerometer-201...|
| 21| 47| 42|

In [74]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

In [75]:
indexer = StringIndexer(inputCol = "class", outputCol = "label")

encoder = OneHotEncoder(inputCol = 'label', outputCol = 'labelVec')

vectorAssembler = VectorAssembler(inputCols = ["x", "y", "z"],
                                 outputCol = "features")

normalizer = Normalizer(inputCol = "features", outputCol = "features_norm", p = 1.0)

In [76]:
from pyspark.ml.classification import LinearSVC

In [77]:
lsvc = LinearSVC(maxIter = 10, regParam = 0.1)

In [78]:
df.createOrReplaceTempView('df')
df_two_class = spark.sql("select * from df where class in ('Use_telephone', 'Standup_chair')")
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [79]:
from pyspark.ml import Pipeline

In [80]:
pipeline = Pipeline(stages = [indexer, encoder, vectorAssembler, normalizer, lsvc])

In [81]:
model = pipeline.fit(df_train)

In [82]:
prediction = model.transform(df_train)

In [83]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [84]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction')
evaluator.evaluate(prediction)

0.9386319573255747

In [85]:
prediction = model.transform(df_test)

In [86]:
evaluator.evaluate(prediction)

0.9396165941798973

# Gradient Boosted Trees with Apache SparkML

In [87]:
configuration_name = 'os_45b77029797b4322b732418ac53a105b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

df = spark.read.parquet(cos.url('hmp.parquet', 'advancedatasci-donotdelete-pr-bvizfkp0o2ej2y'))

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 27| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 23| 51| 38|Brush_teeth|Accelerometer-201...|
| 25| 51| 39|Brush_teeth|Accelerometer-201...|
| 26| 51| 39|Brush_teeth|Accelerometer-201...|
| 25| 52| 39|Brush_teeth|Accelerometer-201...|
| 25| 50| 38|Brush_teeth|Accelerometer-201...|
| 26| 49| 39|Brush_teeth|Accelerometer-201...|
| 24| 48| 39|Brush_teeth|Accelerometer-201...|
| 24| 47| 39|Brush_teeth|Accelerometer-201...|
| 23| 47| 39|Brush_teeth|Accelerometer-201...|
| 20| 48| 39|Brush_teeth|Accelerometer-201...|
| 21| 47| 40|Brush_teeth|Accelerometer-201...|
| 21| 47| 42|

In [88]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [89]:
df.createOrReplaceTempView('df')
df_two_class = spark.sql("select * from df where class in ('Use_telephone', 'Standup_chair')")
splits = df_two_class.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [90]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

In [91]:
indexer = StringIndexer(inputCol = "class", outputCol = "label")

vectorAssembler = VectorAssembler(inputCols = ["x", "y", "z"],
                                 outputCol = "features")

normalizer = Normalizer(inputCol = "features", outputCol = "features_norm", p = 1.0)

In [92]:
from pyspark.ml.classification import GBTClassifier

In [93]:
gbt = GBTClassifier(labelCol = 'label', featuresCol = 'features', maxIter = 10)

In [94]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = [indexer, vectorAssembler, normalizer, gbt])

In [95]:
model = pipeline.fit(df_train)

In [96]:
prediction = model.transform(df_train)

In [97]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("label")

binEval.evaluate(prediction)

0.9112766677954622

In [98]:
prediction = model.transform(df_test)
binEval.evaluate(prediction)

0.8991297953180537

# Hyperparameter-Tuning using GridSeach and CrossValidation in Apache SparkML on Gradient Boosted Trees

In [99]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [100]:
paramGrid = ParamGridBuilder() \
    .addGrid(normalizer.p, [1.0, 2.0, 10.0]) \
    .addGrid(gbt.maxBins, [2, 4, 8, 16]) \
    .addGrid(gbt.maxDepth, [2, 4, 8, 16]) \
    .build()

In [101]:
crossval = CrossValidator(estimator = pipeline,
                         estimatorParamMaps = paramGrid,
                         evaluator = MulticlassClassificationEvaluator(),
                         numFolds = 4)

In [102]:
cvModel = crossval.fit(df_train)

KeyboardInterrupt: 

In [ ]:
prediction = cvModel.transform(df_test)
binEval.evaluate(prediction)

# Using K-Means in Apache SparkML

In [103]:
configuration_name = 'os_45b77029797b4322b732418ac53a105b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

df = spark.read.parquet(cos.url('hmp.parquet', 'advancedatasci-donotdelete-pr-bvizfkp0o2ej2y'))

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 36|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 25| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 27| 51| 37|Brush_teeth|Accelerometer-201...|
| 26| 51| 37|Brush_teeth|Accelerometer-201...|
| 23| 51| 38|Brush_teeth|Accelerometer-201...|
| 25| 51| 39|Brush_teeth|Accelerometer-201...|
| 26| 51| 39|Brush_teeth|Accelerometer-201...|
| 25| 52| 39|Brush_teeth|Accelerometer-201...|
| 25| 50| 38|Brush_teeth|Accelerometer-201...|
| 26| 49| 39|Brush_teeth|Accelerometer-201...|
| 24| 48| 39|Brush_teeth|Accelerometer-201...|
| 24| 47| 39|Brush_teeth|Accelerometer-201...|
| 23| 47| 39|Brush_teeth|Accelerometer-201...|
| 20| 48| 39|Brush_teeth|Accelerometer-201...|
| 21| 47| 40|Brush_teeth|Accelerometer-201...|
| 21| 47| 42|

In [104]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ["x", "y", "z"],
                                 outputCol = "features")

In [105]:
from pyspark.ml.clustering import KMeans

In [106]:
kmeans = KMeans().setK(13).setSeed(1)

In [107]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = [vectorAssembler, kmeans])

In [108]:
model = pipeline.fit(df)

In [109]:
wssse = model.stages[1].computeCost(vectorAssembler.transform(df))
print(wssse)

13737484.905328393


In [110]:
df.createOrReplaceTempView('df')
df = spark.sql("select * from df where class in ('Climb_stairs', 'Brush_teeth')")

In [111]:
model = pipeline.fit(df)

In [112]:
wssse = model.stages[1].computeCost(vectorAssembler.transform(df))
print(wssse)

2618507.395322505
